## Load Dataset

In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm_notebook as tqdm 
import os
import scipy.signal
from tqdm import tqdm_notebook as tqdm
from PIL import Image

In [5]:
## rename files
######## DO ME ONCE THEN SKIP ME

# folder = 'data/test/old/'
# file_list = os.listdir(folder)
# for i in tqdm(range(int(len(file_list)/2))):

#     src, _ = os.path.splitext(file_list[i * 2])
#     dst = f'{i + 1:08}'
    
# #     for ext in ['.jpg', '.txt']:
#     for ext in ['.jpg']:
#         os.rename(folder + src + ext, folder + dst + ext)
    


In [335]:
## Load img #17

# for n_img in range(1, 40000):
n_img = 22
img = plt.imread(f'data/{n_img:08}.jpg') / 255
text = open(f'data/{n_img:08}.txt', encoding='utf-8').readlines()[0]

In [12]:

for n_img in tqdm(range(1, 40078)):
    ## Load img #17

    # for n_img in range(1, 40000):
    img = plt.imread(f'data/train/{n_img:08}.jpg') / 255
    text = open(f'data/train/{n_img:08}.txt', encoding='utf-8').readlines()[0]


    ## processing

    # compute profiles
    v_profile = np.mean(img, axis=1)

    ## not used
    h_profile = np.mean(img, axis=0)

    # smoothing
    def moving_average(a, n=3) :
        ret = np.cumsum(a, dtype=float)
        ret[n:] = ret[n:] - ret[:-n]
        return ret[n - 1:] / n

    v_profile_smooth = moving_average(v_profile, 24)


    v_roi = scipy.signal.argrelextrema(v_profile_smooth, np.greater, order=64)[0]
    v_roi = np.insert(v_roi, 0, 0)
    v_roi = np.append(v_roi, img.shape[0] - 1)

    ## Vertical Pre-Segmentation
    v_delimiters = [(v_roi[i - 1], v_roi[i]) for i in range(1, v_roi.shape[0])]
    v_heights = [(idx[1] - idx[0]) for idx in v_delimiters]
    median_height = np.median(v_heights)



    ## too large??
    for i in range(len(v_heights)):
        if v_heights[i] > median_height * 1.2:
            window = 20
            sub_peaks = scipy.signal.argrelextrema(v_profile_smooth[v_roi[i] + window:v_roi[i + 1] - window], np.less, order=24)
            # if not, problemmm
            if (len(sub_peaks) == 1):
                v_roi = np.append(v_roi, sub_peaks[0] + v_roi[i] - 10)

    v_roi = np.sort(v_roi)



    # resegment
    shift = 8
    v_roi[:-1] += shift
    v_delimiters = [(v_roi[i - 1], v_roi[i]) for i in range(1, v_roi.shape[0])]
    v_heights = [(idx[1] - idx[0]) for idx in v_delimiters]
    median_height = np.median(v_heights)


    ## glue?
    i = 0
    v_delimiters_fresh = []
    segment_is_short = v_heights < median_height

#     print(len(v_roi))
#     print(len(segment_is_short))
#     print(len(v_heights))
#     print(len(v_delimiters))

#     print(v_roi)
#     print(len(v_delimiters))

    while True:

        if i == len(v_heights) - 1:
            # special case, last char
            v_delimiters_fresh.append((v_delimiters[i][0], v_delimiters[i][1]))
        elif (segment_is_short[i]) and (segment_is_short[i + 1]) and (v_heights[i] + v_heights[i + 1] < median_height * 1.7):
            ## NOTE watch for really small segments, characters of 3+ segments

            # glue
            v_delimiters_fresh.append((v_delimiters[i][0], v_delimiters[i + 1][1]))

            # skip one
            i += 1


        else:
            v_delimiters_fresh.append((v_delimiters[i][0], v_delimiters[i][1]))

        ## break condition
        i += 1
        if i >= len(v_heights):
            break

#     print(v_delimiters_fresh)

    ## White spaces
    whities = []
    for i in range(len(v_delimiters_fresh)):
        if np.mean(img[v_delimiters_fresh[i][0]:v_delimiters_fresh[i][1]]) > 0.77:
            whities.append(i)

    for i in np.flip(whities):
        del v_delimiters_fresh[i]

#     print(v_delimiters_fresh)


    ## Horizontal now

    h_delimiters = []
    for d in v_delimiters_fresh:
        h_profile = np.mean(img[d[0]:d[1]], axis=0)
        h_profile = moving_average(h_profile, 4)

        h_cut = np.quantile(h_profile, 0.9)
        h_profile_bin = np.array(h_profile < h_cut, dtype=int)

    #     print(h_profile_bin)

        # mid
        h_mid = int(h_profile.shape[0] / 2)

        # <--|
        h_roi_left = h_mid
        while h_roi_left > 1:
            if h_profile_bin[h_roi_left - 1] == 0:
                break
            else:
                h_roi_left -= 1
        # |-->
        h_roi_right = h_mid
        while h_roi_right < h_profile_bin.shape[0] - 1:
            if h_profile_bin[h_roi_right + 1] == 0:
                break
            else:
                h_roi_right += 1


        h_delimiters.append((h_roi_left, h_roi_right))




    # PLOTSSSS
#     fig, ax = plt.subplots(1,2)
#     fig.set_size_inches((16, 12))

#     ax[0].plot(v_profile_smooth)
#     ax[0].plot([v_cut] * v_profile.shape[0])
#     ax[0].plot(v_roi[:-1] - shift, v_profile_smooth[v_roi[:-1] - shift], 'rx')
#     ax[1].plot(h_profile)
#     ax[1].plot([h_cut] * h_profile.shape[0])


#     fig.show()

#     # boxes
#     plt.figure(figsize=(img.shape[0]//10, img.shape[1]//10))
#     plt.imshow(img, cmap='gray')


#     for i in range(len(v_delimiters_fresh)):
#         # V
#         plt.plot(np.arange(img.shape[1]), [v_delimiters_fresh[i]] * img.shape[1], 'r')

#         # H
#         plt.plot([h_delimiters[i][0]] * (v_delimiters_fresh[i][1] - v_delimiters_fresh[i][0]),np.arange(v_delimiters_fresh[i][0], v_delimiters_fresh[i][1]), 'r')
#         plt.plot([h_delimiters[i][1]] * (v_delimiters_fresh[i][1] - v_delimiters_fresh[i][0]), np.arange(v_delimiters_fresh[i][0], v_delimiters_fresh[i][1]), 'r')


#     plt.show()

#     print(text)


    stripe_length = len(v_delimiters_fresh)

    ## save

    for j in range(min([stripe_length, len(text)])):
        ## NOTE check vs text length for mismatches?
        target = '.\\cropped\\train\\' + text[j]

        try:
            os.mkdir(target)
        except FileExistsError:
            pass
        
        char_img = img[v_delimiters_fresh[j][0]:v_delimiters_fresh[j][1], h_delimiters[j][0]: h_delimiters[j][1]]

        im = Image.fromarray(np.uint8(char_img*255))
        try:
            im.save(f'{target}\\{n_img}_{j}.jpg', "JPEG")
        except:
            print("##### ERRRR")

##### ERRRR


C:\Users\R7008E\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\R7008E\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\R7008E\Anaconda3\lib\site-packages\numpy\core\_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\Users\R7008E\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)
C:\Users\R7008E\Anaconda3\lib\site-packages\ipykernel_launcher.py:117: RuntimeWarning: invalid value encountered in less


##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
####

##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
####

##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
##### ERRRR
####

## Segment Images

## Model

In [1]:
from keras.models import Model
from keras.layers import (
        Input,
        Flatten,
        Dense,
        ZeroPadding2D,
        Conv2D,
        Activation,
        MaxPooling2D,
        BatchNormalization)
from keras.layers.advanced_activations import LeakyReLU

# https://github.com/angzhou/anchor

Using TensorFlow backend.
